<a href="https://colab.research.google.com/github/xiner-hash/feature_engineering-/blob/main/(04.21)%20Programming_Course_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from numpy import loadtxt

from sklearn import tree
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import statsmodels.api as sm

from statistics import *

from xgboost import XGBClassifier

from matplotlib import pyplot

from sklearn import tree, metrics
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score

from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Time-related

In [108]:
"""
action = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')
affect = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')
submission = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')
user = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/user.csv',delimiter=';')
user['pre_test'] = pd.to_numeric(user['pre_test'].str.replace(',', ''))
user['mid_test'] = pd.to_numeric(user['mid_test'].str.replace(',', ''))
"""

"\naction = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')\naffect = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')\nsubmission = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')\nuser = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/user.csv',delimiter=';')\nuser['pre_test'] = pd.to_numeric(user['pre_test'].str.replace(',', ''))\nuser['mid_test'] = pd.to_numeric(user['mid_test'].str.replace(',', ''))\n"

In [109]:
df = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/submission.csv',delimiter=';')

In [110]:
df

,task_submit_id,user_id,task_id,score,upload_date,error_count,test_count
0,207283,1641,244,0.0,2022-10-14 02:15:11,0,1
1,207284,1797,244,1.0,2022-10-14 02:46:29,0,1
2,207285,1797,245,1.0,2022-10-14 02:47:36,0,1
3,207286,1797,246,0.0,2022-10-14 02:48:44,1,0
4,207287,1797,246,1.0,2022-10-14 02:49:00,0,1
...,...,...,...,...,...,...,...
83822,291785,1681,347,0.0,2023-02-04 18:31:54,0,10
83823,291786,1681,347,0.0,2023-02-04 18:33:11,0,10
83824,291787,1681,347,0.3,2023-02-04 18:36:47,0,10
83825,291788,1681,347,0.0,2023-02-04 18:37:51,0,10


In [111]:
# convert upload_date column to datetime format
df['upload_date'] = pd.to_datetime(df['upload_date'])


df['time_of_day'] = pd.cut(df['upload_date'].dt.hour,
                           bins=[6, 12, 18, 24],
                           labels=['Morning', 'Afternoon', 'Evening'],
                           include_lowest=False,
                           ordered=False)


# generate Extreme Time of Day feature
df['extreme_time_of_day'] = pd.cut(df['upload_date'].dt.hour,
                                   bins=[0, 4, 8, 21, 24],
                                   labels=['late_night', 'early_morning', 'non_extreme_hours', 'late_night'],
                                   ordered=False)

# generate Day of Week feature
df['day_of_week'] = df['upload_date'].dt.day_name()
df['day_of_week'] = df['day_of_week'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')


# Create dummy variables
time_of_day_dummies = pd.get_dummies(df['time_of_day'], prefix='time_of_day')
extreme_time_of_day_dummies = pd.get_dummies(df['extreme_time_of_day'], prefix='extreme_time_of_day')
day_of_week_dummies = pd.get_dummies(df['day_of_week'], prefix='day_of_week')

# concatenate dummy variables with original dataframe
df = pd.concat([df, time_of_day_dummies, extreme_time_of_day_dummies, day_of_week_dummies], axis=1)

In [112]:
df = df.drop(['time_of_day', 'extreme_time_of_day','day_of_week'], axis=1)


In [113]:
df = df.drop(["upload_date"], axis=1)

In [114]:
df2 = df

In [115]:
df2

,task_submit_id,user_id,task_id,score,error_count,test_count,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend
0,207283,1641,244,0.0,0,1,0,0,0,0,1,0,1,0
1,207284,1797,244,1.0,0,1,0,0,0,0,1,0,1,0
2,207285,1797,245,1.0,0,1,0,0,0,0,1,0,1,0
3,207286,1797,246,0.0,1,0,0,0,0,0,1,0,1,0
4,207287,1797,246,1.0,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,291785,1681,347,0.0,0,10,0,1,0,0,0,1,0,1
83823,291786,1681,347,0.0,0,10,0,1,0,0,0,1,0,1
83824,291787,1681,347,0.3,0,10,0,1,0,0,0,1,0,1
83825,291788,1681,347,0.0,0,10,0,1,0,0,0,1,0,1


## Percentage Correctness

In [116]:
# df = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/action_log.csv',delimiter=';')


### Chunk Dataset

In [117]:
# Chunk Dataset to task-level 

df["task_status"] = 0

df["user_id"] = df["user_id"].astype("string")
df["task_id"] = df["task_id"].astype("string")

df["task_change"] = df["task_id"].shift(1, fill_value=df["task_id"].iloc[0]) != df["task_id"]

df.loc[df["task_change"] == True, "task_status"] = True


df["student_change"] = df["user_id"].shift(1, fill_value=df["user_id"].iloc[0]) != df["user_id"]
df.loc[df["student_change"] == True, "task_status"] = True

df["task_status"].value_counts()


df['new_task_id'] = df['task_status'].cumsum()

df['new_task_id'] = df['new_task_id'].astype(int)
df = df.drop(['task_id', 'task_change', 'student_change','task_status'], axis=1) 

df = df.rename(columns={'new_task_id': 'task_id'})



df = df.reset_index(drop=True)
df["task_id"]=df["task_id"]+1

In [118]:
import pandas as pd

# assume the DataFrame containing the submission data is called 'submissions'
submission_counts = df.groupby('user_id')['task_submit_id'].count().reset_index()
submission_counts.columns = ['user_id', 'total_submissions']
submission_counts['rank'] = submission_counts['total_submissions'].rank(method='dense', ascending=False)

# print the top 3 students with the highest total number of submissions
top_students = submission_counts.sort_values('total_submissions', ascending=False).head(3)
print(top_students)

    user_id  total_submissions  rank
136    1723               1742   1.0
165    1756               1639   2.0
1      1372               1298   3.0


In [119]:
df

,task_submit_id,user_id,score,error_count,test_count,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,task_id
0,207283,1641,0.0,0,1,0,0,0,0,1,0,1,0,1
1,207284,1797,1.0,0,1,0,0,0,0,1,0,1,0,2
2,207285,1797,1.0,0,1,0,0,0,0,1,0,1,0,3
3,207286,1797,0.0,1,0,0,0,0,0,1,0,1,0,4
4,207287,1797,1.0,0,1,0,0,0,0,1,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,291785,1681,0.0,0,10,0,1,0,0,0,1,0,1,55852
83823,291786,1681,0.0,0,10,0,1,0,0,0,1,0,1,55852
83824,291787,1681,0.3,0,10,0,1,0,0,0,1,0,1,55852
83825,291788,1681,0.0,0,10,0,1,0,0,0,1,0,1,55852


In [120]:
# Create new columns for the desired features


# Define a function to calculate the rolling sum of the last n elements in a series
def rolling_sum_last_n(series, n):
    return series.rolling(n).sum().shift(0)

# Calculate the rolling sum of the last 3 incorrect attempts for each user
df["last_3_attempts_incorrect"] = df.groupby("task_id")["error_count"].apply(lambda x: rolling_sum_last_n(x, 3)).reset_index(level=0, drop=True)
df["last_5_attempts_incorrect"] = df.groupby("task_id")["error_count"].apply(lambda x: rolling_sum_last_n(x, 5)).reset_index(level=0, drop=True)
df["last_8_attempts_incorrect"] = df.groupby("task_id")["error_count"].apply(lambda x: rolling_sum_last_n(x, 8)).reset_index(level=0, drop=True)

df["last_3_attempts_correct"] = df.groupby("task_id")["test_count"].apply(lambda x: rolling_sum_last_n(x, 3)).reset_index(level=0, drop=True)
df["last_5_attempts_correct"] = df.groupby("task_id")["test_count"].apply(lambda x: rolling_sum_last_n(x, 5)).reset_index(level=0, drop=True)
df["last_8_attempts_correct"] = df.groupby("task_id")["test_count"].apply(lambda x: rolling_sum_last_n(x, 8)).reset_index(level=0, drop=True)


# Replace NaN values with 0
df.fillna(0, inplace=True)

<ipython-input-120-82e0a050f17c>:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df["last_3_attempts_incorrect"] = df.groupby("task_id")["error_count"].apply(lambda x: rolling_sum_last_n(x, 3)).reset_index(level=0, drop=True)
<ipython-input-120-82e0a050f17c>:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group

In [121]:
#df = pd.merge(df, df2, on='task_submit_id')

## Attempt Count

In [122]:
# create new columns for the features
df['cumulative_correct'] = 0
df['cumulative_incorrect'] = 0

# initialize variables to keep track of correct and incorrect answers
correct_count = 0
incorrect_count = 0
current_task_id = None

# loop through the rows of the DataFrame
for i, row in df.iterrows():
    # if the task_id changed, reset the counts
    if row['task_id'] != current_task_id:
        correct_count = 0
        incorrect_count = 0
        current_task_id = row['task_id']
    
    # update the counts based on the test_succeeded
    if row['error_count'] != 0:
        incorrect_count += row['error_count']
    else:
        correct_count += row['test_count']
    
    # update the new columns in the DataFrame
    df.at[i, 'cumulative_correct'] = correct_count
    df.at[i, 'cumulative_incorrect'] = incorrect_count


In [123]:
df

,task_submit_id,user_id,score,error_count,test_count,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,...,day_of_week_Weekend,task_id,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect
0,207283,1641,0.0,0,1,0,0,0,0,1,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,207284,1797,1.0,0,1,0,0,0,0,1,...,0,2,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,207285,1797,1.0,0,1,0,0,0,0,1,...,0,3,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,207286,1797,0.0,1,0,0,0,0,0,1,...,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,207287,1797,1.0,0,1,0,0,0,0,1,...,0,4,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,291785,1681,0.0,0,10,0,1,0,0,0,...,1,55852,0.0,0.0,0.0,30.0,50.0,80.0,450,49
83823,291786,1681,0.0,0,10,0,1,0,0,0,...,1,55852,0.0,0.0,0.0,30.0,50.0,80.0,460,49
83824,291787,1681,0.3,0,10,0,1,0,0,0,...,1,55852,0.0,0.0,0.0,30.0,50.0,80.0,470,49
83825,291788,1681,0.0,0,10,0,1,0,0,0,...,1,55852,0.0,0.0,0.0,30.0,50.0,80.0,480,49


In [124]:
df = df.drop(["test_count",'error_count',"task_id",'score'], axis=1)

## Chunk Data and Aggregate Feature

In [125]:
df2 = pd.read_csv('/content/drive/MyDrive/Research/Programming/2022/affect.csv',delimiter=';')

In [126]:
df2

,user_id,task_id,action_date,task_submit_id,affect
0,1584,378,2022-10-12 16:33:53,207148,OT
1,1582,372,2022-10-12 19:02:27,207153,AN
2,1582,372,2022-10-12 19:13:33,207160,BO
3,1582,372,2022-10-12 20:11:54,207162,BO
4,1573,372,2022-10-13 05:34:56,207176,FO
...,...,...,...,...,...
21406,1672,290,2023-01-26 18:14:14,291661,FO
21407,1672,290,2023-01-26 18:14:46,291662,FR
21408,1672,29,2023-01-26 18:59:09,291667,BO
21409,1674,45,2023-01-26 19:38:50,291681,FO


In [127]:
df2 = df2.drop(["action_date","user_id"], axis=1)


In [128]:
df = pd.merge(df, df2, on='task_submit_id')


In [129]:
df

,task_submit_id,user_id,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect,task_id,affect
0,207314,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,372,OT
1,207315,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,372,FO
2,207317,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,372,FO
3,207324,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,379,FO
4,207326,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,379,FO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21352,291661,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,24.0,0.0,0.0,24,0,290,FO
21353,291662,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,24.0,0.0,0.0,32,0,290,FR
21354,291667,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,29,BO
21355,291681,1674,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,45,FO


### Rechunk Dataset

In [130]:
df["task_status"] = 0

df["user_id"] = df["user_id"].astype("string")
df["task_id"] = df["task_id"].astype("string")

df["task_change"] = df["task_id"].shift(1, fill_value=df["task_id"].iloc[0]) != df["task_id"]

df.loc[df["task_change"] == True, "task_status"] = True


df["student_change"] = df["user_id"].shift(1, fill_value=df["user_id"].iloc[0]) != df["user_id"]
df.loc[df["student_change"] == True, "task_status"] = True

df["task_status"].value_counts()


df['new_task_id'] = df['task_status'].cumsum()

df['new_task_id'] = df['new_task_id'].astype(int)
df = df.drop(['task_id', 'task_change', 'student_change','task_status'], axis=1) 

df = df.rename(columns={'new_task_id': 'task_id'})



df = df.reset_index(drop=True)
df["task_id"]=df["task_id"]+1

In [131]:
df

,task_submit_id,user_id,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect,affect,task_id
0,207314,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,OT,1
1,207315,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,FO,1
2,207317,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,FO,1
3,207324,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,FO,2
4,207326,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,FO,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21352,291661,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,24.0,0.0,0.0,24,0,FO,12802
21353,291662,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,24.0,0.0,0.0,32,0,FR,12802
21354,291667,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,BO,12803
21355,291681,1674,0,0,1,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,FO,12804


In [132]:
unique_affect = df['affect'].unique()
print(unique_affect)

affect_counts = df['affect'].value_counts()
print(affect_counts)


['OT' 'FO' 'BO' 'AN' 'FR' 'CO']
FO    11842
FR     2702
OT     2271
AN     1911
CO     1468
BO     1163
Name: affect, dtype: int64


In [133]:
df  = df.groupby("task_id").last()


In [134]:
features = df.loc[:, 'time_of_day_Morning':'cumulative_incorrect']

# SumattemptCount (no time) or time_of_day_Morning

features.fillna(features.mean(), inplace=True)
students = df['user_id'].values.tolist()

In [135]:
affect_labels = ['AN','FO','OT','BO','CO','FR']
affect_label = df['affect']

In [136]:
df

,task_submit_id,user_id,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,extreme_time_of_day_early_morning,extreme_time_of_day_late_night,extreme_time_of_day_non_extreme_hours,day_of_week_Weekday,day_of_week_Weekend,last_3_attempts_incorrect,last_5_attempts_incorrect,last_8_attempts_incorrect,last_3_attempts_correct,last_5_attempts_correct,last_8_attempts_correct,cumulative_correct,cumulative_incorrect,affect
task_id,,,,,,,,,,,,,,,,,,,
1,207317,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,FO
2,207329,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,FO
3,207330,1645,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,FO
4,207333,1649,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,FO
5,207343,1645,0,0,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,OT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12801,291652,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,14,0,AN
12802,291662,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,24.0,0.0,0.0,32,0,FR
12803,291667,1672,0,1,0,0,0,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,BO


In [137]:
import pandas as pd

# assume the DataFrame containing the submission data is called 'submissions'
submission_counts = df.groupby('user_id')['task_submit_id'].count().reset_index()
submission_counts.columns = ['user_id', 'total_submissions']
submission_counts['rank'] = submission_counts['total_submissions'].rank(method='dense', ascending=False)

# print the top 3 students with the highest total number of submissions
top_students = submission_counts.sort_values('total_submissions', ascending=False).head(3)
print(top_students)

    user_id  total_submissions  rank
118    1723                284   1.0
62     1656                213   2.0
184    1806                211   3.0


In [138]:
df = df.drop(['task_submit_id'], axis=1)


In [ ]:
df

# Model Building 

### **Random Forest**

In [140]:
# 5-fold student-level cross validation 

group_dict = {}
groups = np.array([])
for index, row in df.iterrows():
    student_id = row['user_id']
    if student_id not in group_dict:
        group_dict[student_id] = index
    groups = np.append(groups, group_dict[student_id])
gkf = GroupKFold(n_splits=5)

In [141]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = RandomForestClassifier(n_estimators=100,random_state=1)

        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5014274121359493
Affective label:  FO
AUC ROC performance:  0.5199564207968912
Affective label:  OT
AUC ROC performance:  0.5035210496524126
Affective label:  BO
AUC ROC performance:  0.49973563613939476
Affective label:  CO
AUC ROC performance:  0.5006418564813833
Affective label:  FR
AUC ROC performance:  0.5032319713247562


### **ANN**

In [142]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = MLPClassifier(random_state=1,solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 3))


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5
Affective label:  FO
AUC ROC performance:  0.5065150895832815


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Affective label:  OT
AUC ROC performance:  0.5
Affective label:  BO
AUC ROC performance:  0.5
Affective label:  CO
AUC ROC performance:  0.5
Affective label:  FR
AUC ROC performance:  0.5


### **Decision Tree**

In [143]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = DecisionTreeClassifier(random_state=1,max_leaf_nodes= None, min_samples_leaf= 2, min_samples_split=10)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5005758877052109
Affective label:  FO
AUC ROC performance:  0.5244595045495286
Affective label:  OT
AUC ROC performance:  0.5005386339989022
Affective label:  BO
AUC ROC performance:  0.5003238997630826
Affective label:  CO
AUC ROC performance:  0.5001445439545658
Affective label:  FR
AUC ROC performance:  0.5053147189482734


### **SVM**

In [144]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = svm.SVC(kernel='rbf', decision_function_shape='ovr',random_state = 1)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5
Affective label:  FO
AUC ROC performance:  0.5101298280660631
Affective label:  OT
AUC ROC performance:  0.5
Affective label:  BO
AUC ROC performance:  0.5
Affective label:  CO
AUC ROC performance:  0.5
Affective label:  FR
AUC ROC performance:  0.5


### **Naive Bayes**

In [145]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = GaussianNB(priors=[0.1, 0.9])


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))




Affective label:  AN
AUC ROC performance:  0.5079361559820276
Affective label:  FO
AUC ROC performance:  0.5031133887447974
Affective label:  OT
AUC ROC performance:  0.5043971360206256
Affective label:  BO
AUC ROC performance:  0.5087106769597508
Affective label:  CO
AUC ROC performance:  0.4940138135304556
Affective label:  FR
AUC ROC performance:  0.49188052380661273


### **XGB**

In [146]:
k_fold = GroupKFold(n_splits=5)

for x in affect_labels:
    label = affect_label.mask(affect_label.ne(x), 'Not ' + x, inplace = False)
    #print(x,label)

    training_scores = []
    validation_scores = []
    aucroc = []

    for train_index, test_index in k_fold.split(features.values.tolist(),
                                                label.values.tolist(),
                                                groups=students):
        X_train = features.iloc[train_index]
        X_test = features.iloc[test_index]
        y_train = label.iloc[train_index]
        y_test = label.iloc[test_index]
        y_train = pd.get_dummies(y_train)[x]
        y_test = pd.get_dummies(y_test)


        clf = XGBClassifier(learning_rate=0.05, n_estimators=50, random_state=1)


        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        #training_scores.append(clf.score(X_train, y_train))
        #validation_scores.append(clf.score(X_test, y_test[x]))
        aucroc.append(roc_auc_score(y_test[x], pred))

    print("Affective label: ", x)
    #print("Training performance: ", np.mean(training_scores))
    # print("Testing performance: ", np.mean(validation_scores))
    print("AUC ROC performance: ", np.mean(aucroc))



Affective label:  AN
AUC ROC performance:  0.5
Affective label:  FO
AUC ROC performance:  0.5361054310122125
Affective label:  OT
AUC ROC performance:  0.49973687738385825
Affective label:  BO
AUC ROC performance:  0.49987873888439777
Affective label:  CO
AUC ROC performance:  0.4998734241680484
Affective label:  FR
AUC ROC performance:  0.5010924867601612
